In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
features = pd.read_csv('public_data_1/mimic_synthetic_feat.name',header=None).values.flatten()
labels = pd.read_csv('public_data_1/mimic_synthetic_label.name', header=None).values.flatten()
X_full = pd.read_csv('public_data_1/mimic_synthetic_train.data', header=None, names=features, sep=' ')
y = pd.Series(pd.read_csv('public_data_1/mimic_synthetic_train.solution', header=None, names=labels, sep=',').values.flatten())

In [3]:
types = pd.read_csv('public_data_1/mimic_synthetic_feat.type', header=None).set_index(features)[0]
print(types.value_counts())

Numerical      226
Binary         107
Categorical      8
Name: 0, dtype: int64


In [4]:
X = X_full.drop(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME'], axis=1)
X = pd.get_dummies(X, columns=['INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'ICU', 'GENDER'])
X = X.fillna(0)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1234)

In [8]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import confusion_matrix

clf_lasso = LassoCV(cv=4).fit(X_train, y_train)
y_pred = clf_lasso.predict(X_test)
y_pred[y_pred >= 0] = 1
y_pred[y_pred < 0] = 0
C = confusion_matrix(y_test, y_pred)
per_class = np.diag(C) / C.sum(axis=1)
score = np.mean(per_class)
print(score)

0.6005902573875954


In [14]:
test = pd.read_csv('public_data_1/mimic_synthetic_test.data', header=None, names=features, sep=' ')
X_test = test.drop(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME'], axis=1)
X_test = pd.get_dummies(X_test, columns=['INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'ICU', 'GENDER'])
X_test = X_test.fillna(0)
X_test = X_test.reindex(columns = X.columns, fill_value=0)
prediction = clf_lasso.predict(X_test)
pd.Series(prediction).to_csv('classification_aibek.csv', index=False)